In [48]:
# 셀레니움으로 크롤링
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException
from bs4 import BeautifulSoup
from io import StringIO
import pandas as pd
import time

# 1. Chrome Driver & Options 설정
options = Options()
options.binary_location = ".\\chrome-win64\\chrome.exe"
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--window-size=1920,1080")
service = Service("chromedriver.exe")
driver = webdriver.Chrome(service=service, options=options)
wait = WebDriverWait(driver, 10)

# 2. KBO 홈페이지 접속 및 선수 조회 페이지 이동 셀레니움
driver.get("https://www.koreabaseball.com/Default.aspx?vote=true")
hover = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "#lnb > li:nth-child(3) > a")))
ActionChains(driver).move_to_element(hover).perform()
player_menu = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#lnb > li:nth-child(3) > div > ul > li:nth-child(1) > a")))
player_menu.click()

# 3. 팀/포지션 드롭다운 추출
team_select = Select(wait.until(EC.presence_of_element_located((By.ID, "cphContents_cphContents_cphContents_ddlTeam"))))
team_list = [(i, option.text) for i, option in enumerate(team_select.options) if i != 0] # 0번 빼고 실행

print("[팀 목록]", [t[1] for t in team_list])

df_all = []

# 4. 팀별 크롤링
for team_index, team_name in team_list:
    print(f"\n▶ 팀: {team_name}")
    
    # 드롭다운 다시 로드해야 함
    team_select = Select(wait.until(EC.presence_of_element_located((By.ID, "cphContents_cphContents_cphContents_ddlTeam"))))
    team_select.select_by_index(team_index) # 숫자 선택
    time.sleep(1)

    search_btn = wait.until(EC.element_to_be_clickable((By.ID, "cphContents_cphContents_cphContents_btnSearch")))
    driver.execute_script("arguments[0].click();", search_btn)  # 브라우저 안에서 직접 자바스크립트로 실행해 클릭(클릭 잘 안돼서 집어넣음)
    time.sleep(2)

    page = 1
    while True:
        soup = BeautifulSoup(driver.page_source, "html.parser")
        table = soup.find("table", class_="tEx")

        try:
            df = pd.read_html(StringIO(str(table)))[0]
            df["TEAM_NAME"] = team_name
            df_all.append(df)
        except:
            print(f"⚠️ 표 없음: {team_name} - 페이지 {page}")
            break

        # 다음 페이지 시도
        next_id = f"cphContents_cphContents_cphContents_ucPager_btnNo{page + 1}"
        try:
            next_button = driver.find_element(By.ID, next_id)
            driver.execute_script("arguments[0].click();", next_button)
            time.sleep(2)
            page += 1
        except NoSuchElementException:
            break

driver.quit()

# 5. 데이터프레임 병합 및 확인
final_df = pd.concat(df_all, ignore_index=True)
print("\n✅ 전체 크롤링 완료!")
final_df

[팀 목록] ['한화', 'LG', '롯데', 'KT', '삼성', 'SSG', 'KIA', 'NC', '두산', '키움']

▶ 팀: 한화

▶ 팀: LG

▶ 팀: 롯데

▶ 팀: KT

▶ 팀: 삼성

▶ 팀: SSG

▶ 팀: KIA

▶ 팀: NC

▶ 팀: 두산

▶ 팀: 키움

✅ 전체 크롤링 완료!


,등번호,선수명,팀명,포지션,생년월일,체격,출신교,TEAM_NAME
0,17.0,권광민,한화,내야수,1997-12-12,"189cm, 102kg",서울청구초-홍은중-장충고,한화
1,64.0,권민규,한화,투수,2006-05-13,"188cm, 90kg",청주석교초-세광중-세광고,한화
2,56.0,김건,한화,내야수,2000-02-23,"183cm, 79kg",양정초-개성중-경남고,한화
3,NaN,김규연,한화,투수,2002-08-23,"183cm, 91kg",동수원초(수원영통구리틀)-매향중-공주고-한화-상무,한화
4,15.0,김기중,한화,투수,2002-11-16,"186cm, 96kg",의왕부곡초-매송중-유신고,한화
...,...,...,...,...,...,...,...,...
896,30.0,주승우,키움,투수,2000-01-30,"180cm, 82kg",송추초(의정부리틀)-영동중-서울고-성균관대,키움
897,53.0,최주환,키움,내야수,1988-02-28,"177cm, 73kg",학강초-광주동성중-광주동성고-두산-상무-두산-SK-SSG,키움
898,4.0,카디네스,키움,외야수,1997-10-10,"185cm, 93kg",미국 California State(대)-삼성,키움
899,NaN,푸이그,키움,외야수,1990-12-07,"188cm, 108kg",쿠바 EIEFD(대)-키움,키움


In [49]:
# final_df 테이블 수정
# 1. player_id 컬럼 추가 & 번호 1부터901까지 출력되게 변경
final_df.insert(0, "PLAYER_ID", pd.Series(range(1, len(final_df) + 1), dtype=int))
                
# 2. 디비에 맞게 컬럼명 변경
final_df.rename(columns = {
    "등번호" : "PLAYER_BACK_NUMBER",
    "선수명" : "PLAYER_NAME",
    "팀명" : "TEAM_ID",
    "포지션" : "PLAYER_POSITION",
    "생년월일" : "PLAYER_BIRTH_DATE",
    "체격" : "PLAYER_HEIGHT_WEIGHT",
    "출신교" : "PLAYER_EDUCATION_PATH"
}, inplace=True)

# 3. 컬럼 순서 변경
final_df = final_df[
    ["PLAYER_ID", "PLAYER_NAME", "PLAYER_POSITION", "TEAM_ID",
     "PLAYER_BACK_NUMBER", "PLAYER_BIRTH_DATE", "PLAYER_HEIGHT_WEIGHT", "PLAYER_EDUCATION_PATH"]
]

# 4. 팀명을 숫자로 치환
team_name_to_id = {
    "한화": 9, "LG": 5, "롯데": 2, "KT": 7,
    "삼성": 3, "SSG": 8, "KIA": 4, "NC": 1,
    "두산" : 6, "키움" : 10, "고양" : 11
}
final_df["TEAM_ID"] = final_df["TEAM_ID"].replace(team_name_to_id)
final_df["TEAM_ID"] = final_df["TEAM_ID"].astype(int)

C:\Users\Admin\AppData\Local\Temp\ipykernel_8468\3653823.py:28: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  final_df["TEAM_ID"] = final_df["TEAM_ID"].replace(team_name_to_id)
C:\Users\Admin\AppData\Local\Temp\ipykernel_8468\3653823.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df["TEAM_ID"] = final_df["TEAM_ID"].replace(team_name_to_id)
C:\Users\Admin\AppData\Local\Temp\ipykernel_8468\3653823.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc

In [76]:
# 디비 형식에 맞게 null 값 변경
# 1. 등번호는 int64타입이라 object로 바꿔줘야 none으로 바뀜
final_df["PLAYER_BACK_NUMBER"] = final_df["PLAYER_BACK_NUMBER"].astype("object")
final_df["PLAYER_BACK_NUMBER"] = final_df["PLAYER_BACK_NUMBER"].where(pd.notnull(final_df["PLAYER_BACK_NUMBER"]), None)
# 2. 포지션도 변경
final_df["PLAYER_POSITION"] = final_df["PLAYER_POSITION"].where(pd.notnull(final_df["PLAYER_POSITION"]), None)

C:\Users\Admin\AppData\Local\Temp\ipykernel_8468\3310604975.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df["PLAYER_BACK_NUMBER"] = final_df["PLAYER_BACK_NUMBER"].astype("object")
C:\Users\Admin\AppData\Local\Temp\ipykernel_8468\3310604975.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df["PLAYER_BACK_NUMBER"] = final_df["PLAYER_BACK_NUMBER"].where(pd.notnull(final_df["PLAYER_BACK_NUMBER"]), None)
C:\Users\Admin\AppData\Local\Temp\ipykernel_8468\3310604975.py:3: SettingWithCopyWarnin

In [79]:
# nan -> none으로 바꼈는지 확인
print(repr(final_df.loc[3, "PLAYER_BACK_NUMBER"]))       # 출력: None (이게 되어야 정상)
print(repr(final_df.loc[153, "PLAYER_POSITION"]))       # 출력: None (이게 되어야 정상)
print(type(final_df.loc[3, "PLAYER_BACK_NUMBER"]))       # 출력: <class 'NoneType'>

None
None
<class 'NoneType'>


In [80]:
# csv로 저장
final_df.to_csv("팀별_선수.csv", encoding="utf-8-sig")

In [82]:
# PLAYERS 테이블에 데이터 집어넣기
import cx_Oracle as oci
import pandas as pd

# DB 연결
sid = 'XE'
host = '210.119.14.76'
port = 1521
username ='baseball'
password = 'bb1234'

def get_connection():
    conn = oci.connect(f'{username}/{password}@{host}:{port}/{sid}')
    return conn

# 디비에 csv 데이터 삽입
# 1. CSV 불러오기
df = pd.read_csv("팀별_선수.csv", encoding="utf-8-sig")

# 2. Oracle 연결
conn = get_connection()
cursor = conn.cursor()

# 3. INSERT 문 실행 (컬럼명 일치하는지 확인!)
insert_sql = """
    INSERT INTO PLAYERS (
        PLAYER_ID, PLAYER_NAME, PLAYER_POSITION, TEAM_ID,
        PLAYER_BACK_NUMBER, PLAYER_BIRTH_DATE, PLAYER_HEIGHT_WEIGHT, PLAYER_EDUCATION_PATH
    ) VALUES (
        :1, :2, :3, :4, :5, TO_DATE(:6, 'YYYY-MM-DD'), :7, :8
    )
"""

# 4. 데이터 삽입
for i, row in final_df.iterrows():
    try:
        cursor.execute(insert_sql, (
            row.PLAYER_ID,
            row.PLAYER_NAME,
            row.PLAYER_POSITION,
            row.TEAM_ID,
            row.PLAYER_BACK_NUMBER,
            row.PLAYER_BIRTH_DATE,
            row.PLAYER_HEIGHT_WEIGHT,
            row.PLAYER_EDUCATION_PATH
        ))
    except Exception as e:
        print(f"[❌ 에러 발생] index={i}")
        print(f"PLAYER_ID: {row.PLAYER_ID}, BACK_NUMBER: {row.PLAYER_BACK_NUMBER}, type: {type(row.PLAYER_BACK_NUMBER)}")
        print(f"전체 값: {row.to_dict()}")
        print(f"에러: {e}")
        break


# 5. 커밋 & 종료
conn.commit()
cursor.close()
conn.close()

In [12]:
final_df

,PLAYER_ID,PLAYER_NAME,PLAYER_POSITION,TEAM_ID,PLAYER_BACK_NUMBER,PLAYER_BIRTH_DATE,PLAYER_HEIGHT_WEIGHT,PLAYER_EDUCATION_PATH
0,1.0,권광민,내야수,9.0,17.0,1997-12-12,"189cm, 102kg",서울청구초-홍은중-장충고
1,2.0,권민규,투수,9.0,64.0,2006-05-13,"188cm, 90kg",청주석교초-세광중-세광고
2,3.0,김건,내야수,9.0,56.0,2000-02-23,"183cm, 79kg",양정초-개성중-경남고
3,4.0,김규연,투수,9.0,NaN,2002-08-23,"183cm, 91kg",동수원초(수원영통구리틀)-매향중-공주고-한화-상무
4,5.0,김기중,투수,9.0,15.0,2002-11-16,"186cm, 96kg",의왕부곡초-매송중-유신고
...,...,...,...,...,...,...,...,...
897,898.0,최주환,내야수,10.0,53.0,1988-02-28,"177cm, 73kg",학강초-광주동성중-광주동성고-두산-상무-두산-SK-SSG
898,899.0,카디네스,외야수,10.0,4.0,1997-10-10,"185cm, 93kg",미국 California State(대)-삼성
899,900.0,푸이그,외야수,10.0,NaN,1990-12-07,"188cm, 108kg",쿠바 EIEFD(대)-키움
900,901.0,하영민,투수,10.0,50.0,1995-05-07,"183cm, 74kg",광주수창초-진흥중-진흥고-히어로즈
